In [29]:
#Import Libraries
import math
import numpy as np
import matplotlib.pyplot as plt
from iapws import IAPWS97

# Heat Transport

The equation of the conservation of energy is expressed as:

$$ \Delta \left( H + \frac { u^2 }{2} + gZ \right) = Q - W_s $$
$$ VdP + udu + gdZ + \sum F = 0 $$
$$ VdP + udu + gdZ + \sum F = Q - W_s $$

The rate of heat transfer to the control volume element to the surroundings (Rajput, 1999) is given by:

$$H_{T} = - \frac{\ 2 \pi dL (T_{in} - T_{out}) }
    {\left[\frac{1}{h_{in}r_1} 
    + \frac{ln \left(\frac{r_2}{r_1} \right)} {k_A}
    + \frac{ln \left(\frac{r_3}{r_2} \right)} {k_B}
    + \frac{1}{h_{out}r_3}
    \right]}$$

In [30]:
def convection (h, r):
    """convective heat transport steam and inner pipe"""
    return 1/(h*r)

def conduction (r1, r2, k):
    """conductive heat transport across pipe and insulation"""
    return math.log(r2/r1) / k

#Generic function to calculate Ht for a given pipe length
def heat_transfer(L, t_in, t_out, h_in, h_out, k_a, k_b, r1, r2, r3):
    pi_2 = 2 * math.pi
    numerator = pi_2 * L * (t_in - t_out)
    c1 = convection(h_in, r1)
    c2 = conduction(r1, r2, k_a)
    c3 = conduction(r2, r3, k_b)
    c4 = convection(h_out, r3)
    c_tot = c1 + c2 + c3 + c4
    return -(numerator / c_tot)

#Heat loss along the pipe length with dL units; output in array
def calc_heat_loss(x, t_in, t_out, h_in, h_out, k_a, k_b, r1, r2, r3):
    tot_loss = []
    for i in x:
        loss = heat_transfer(i, t_in, t_out, h_in, h_out, k_a, k_b, r1, r2, r3)
        tot_loss.append(loss)
    tot_loss = np.absolute(tot_loss)*1e-6
    return tot_loss

# Momentum Transport

$$ VdP + udu + gdZ + \sum F = 0 $$

$$ (\frac{\frac{1}{\rho_i}+\frac{1}{\rho_{i-1}}}{2})(p_i - p_{i-1})
    + \frac{u^{2}_{i}-u^{2}_{i-1}}{2}
    + g (Z_i + Z_{i-1})
    + \sum F 
    = 0 $$
    
Assumptions: 
   * $ du = 0 $ for straight pipe
   * Pipe roughness, carbon steel; $ \epsilon $ = 0.000046 m
   * $ g = 9.81 \frac{m}{s^{2}} $
   * $ \rho_i $ from steam table; AIPWS97
   * $ \rho_{i-1} $

# Frictional  loses in mechanical energy balance equation

Frictional loses from the friction in:
* straight pipe (Fanning friction)
* enlargement losses $K_ex$
* contraction losses $K_c$
* losses in fittings and valves $K_f $

$$ \sum F = 4f \frac{\Delta L}{D} \frac{v^{2}}{2}
    + K_{ex} \frac{v^{2}_1}{2}
    + K_c \frac{v^{2}_2}{2}
    + K_f \frac{v^{2}_1}{2} $$
    
If the velocity of fluid is changed in direction or magnitude, additional friction losses occur. This results from additional turbulence which develops beause of vortices and other factors. Method to estimate the losses are discussed below:

In [31]:
"""Function to calculate Skin Friction"""
def reynolds_number (diameter, velocity, density, dynamic_viscosity):
    return (diameter*velocity*density/dynamic_viscosity)

def reynolds_number_test():
    test_re = reynolds_number(0.3353, 2.523, 60.52, 0.000233)
    return (test_re - 219732) < 100

#Option1 to calculate friction factor
def ff_fanning (reynolds_number, relative_roughness):
    B = (37530/reynolds_number)**16
    A = (2.457*math.log(((7/reynolds_number)**0.9 +\
        (0.27*(relative_roughness)))**-1))**16
    f = 2*((8/reynolds_number)**12+(A+B)**-1.5)**(1/12)
    return f

#Option2 for friction factor; iterative solution using colebrook equation
def ff_colebrook(reynolds_number, relative_roughness):
    friction = 0.08 #Starting Friction Factor
    while 1:
        leftF = 1 / friction**0.5 #Solve Left side of Eqn
        rightF = - 4 * math.log10(1.256/(reynolds_number * friction**0.5)+(relative_roughness/3.7)) # Solve Right side of Eqn
        friction = friction - 0.00001 #Change Friction Factor
      #  print(leftF)
      #  print(rightF)
      #  print(friction)
        if (rightF - leftF <= 0): #Check if Left = Right
            break
    return friction

#generic function to calculate skin friction
def skin (fanning_factor, length, diameter, velocity, gc = 1):
    return 4*fanning_factor*length* velocity**2 / (diameter*2*gc)

#main function to calculate skin friction
def f_skin(t_sat, diameter, velocity, relative_roughness, length, gc = 1):
    #convert p_sat()
    t_sat = t_sat + 273.15
    #calculate density and dynamic viscosity from IAPWS 97
    sat_steam=IAPWS97(T=t_sat,x=0)  #saturated steam with known P
    density = sat_steam.rho #* 0.062428
    dynamic_viscosity = sat_steam.mu #* 0.67196897514
    #calculate reynolds number
    n_reynolds = reynolds_number(diameter, velocity, density, dynamic_viscosity)
    #calculate fannings friction factor
    fanning_factor = ff_fanning(n_reynolds, relative_roughness)
    #print (fanning_factor)
    return 4*fanning_factor*length* velocity**2 / (diameter*2*gc)

#ff_fanning(219300, 0.000448)
#f_skin(416.03, 0.762, 24.068, .00000604, 100)

0.0019392683356514282


294.84443578991414

A. __Sudden enlargement losses.__ If the cross section of the pipe enlarges suddenly, it results in additional lossesdue to eddies formed by the jet expanding in the enlarged section. This friction loss can be calculated by the following for turbulent flow in both sections:

$$ h_{ex} = \frac {(v_i - v_2)^{2}}{2 \alpha} 
    = \left(1 - \frac{A_1}{A_2}\right)^{2} \frac{v^{2}_1}{2 \alpha} $$
    
where $h_{ex}$ is the friction loss in $J/kg, K_{ex}$ is the expansion-loss coefficient and $ = (1-A_1/A_2)^{2}, v_1$ is the upstream velocity in the smaller area in $m/s, v_2$ is the downstream velocity and $\alpha = 1.0$

In [32]:
"""Function to calculate friction loss from pipe enlargement / expansion"""
def f_expansion(area_upstream, area_downstream, v, alpha=1, gc = 1): #set gc = 32.174 if english units
    Kex = 1-(area_upstream/area_downstream)
    #print (Kex)
    return Kex * v**2 / (2 * alpha * gc)
#for flow from pipe to tank where area_downstream >>> area_upstream A1/A2 -> 0
#f_expansion (1, 1*10**3, 0.6083)

B. __Sudden contraction losses.__ When the cross section of the pipe is suddenly reduced, the stream cannot flow around the sharp corner, and additional frictional losses due to eddies occur. For turbulent flow, this is given by:

$$ h_c = 0.55 \left( 1- \frac{A_2}{A_1} \right) \frac{v^{2}_2}{2 \alpha}  
    = K_c \frac{v^{2}_2} {2 \alpha} \frac{J}{kg}  $$
    
where $h_c$ is the friction loss, $\alpha = 1.0$ for turbulent flow, $v_2$ is the average velocity in the smaller or downstream section, and $K_c$ is the contraction-loss coefficient (P1) and approximately equals $0.55(1-A_2/A_1)$. For laminar flow, the same equation can be used with $\alpha = \frac{1}{2}(S2)$.

In [33]:
"""Function to calculate friction loss from pipe contraction"""
def f_contraction(area_upstream, area_downstream, v, alpha = 1, gc = 1): #set gc = 32.174 if english units
    Kc = 0.55 * (1- (area_downstream / area_upstream))
    return Kc * v**2 / (2 * alpha * gc)
#for flow from pipe to tank where area_upstream >>> area_downstream A2/A1 -> 0
#f_contraction (0.0884, 0.02330, 9.57, gc = 32.174) #0.575

C. __Losses in fittings and valves.__ Pipe fittings and valves also disturb the normal flow lines in a pipe and cause additional friction losses. In a short pipe with many fittings, the friction loss from these fittings could be greater than in the straight pipe. The friction loss for fittings and valves is given by the following equation:
$$ h_f = K_f \frac{v^{2}_1}{2}  $$

where $K_f$ is the loss factor for the fitting or valve and $v_i$ is the average velocity in the pipe leading to the fitting. Experimental values for $K_f$ are given in the python dictionary "Kf" for turbulent flow. For english units, the right side is divided by $g_c$.

In [34]:
"""Dictionary for Friction Loss; Turbulent Flow Through Valves and Fittings"""
Kf = {'elbow45': 0.45, 'elbow90': 0.75, 'tee': 1, 'return_bend': 1.5, \
     'coupling': 0.04, 'union': 0.04, 'gate_valve_full': 0.17, \
     'gate_valve_half': 4.5}

#Generic function for calculating friction loss thru fittings and valves
def fittings (n, Kf, v, gc=1):
    return n* Kf * v**2 * 0.5 / gc


"""Total friction loss from fittings and valves
-->h_f is a function of Kf and velocity only
-->The function will require only velocity as input value; 
default value for number fittings and valves will be provided"""    
#def f_fittings (v, n_elbow45 = 0, n_elbow90 = 1, n_tee = 0, n_return_bend = 0, n_coupling = 0, n_union = 0, \
#                n_gate_valve_full = 0, n_gate_valve_half = 0, gc = 1): #set gc = 32.174 if english units
#    Kf = {'elbow45': 0.45, 'elbow90': 0.75, 'tee': 1, 'return_bend': 1.5, \
#         'coupling': 0.04, 'union': 0.04, 'gate_valve_full': 0.17, 'gate_valve_half': 4.5}
#    #f_elbow90= n_elbow90* Kf['elbow90'] * v**2 * 0.5 / gc
#    f_elbow45 = fittings(n_elbow45, Kf['elbow45'], v, gc)
#    f_elbow90 = fittings(n_elbow90, Kf['elbow90'], v, gc)
#    f_tee = fittings(n_tee, Kf['tee'], v, gc)
#    f_return_bend = fittings(n_return_bend, Kf['return_bend'], v, gc)
#    f_coupling = fittings(n_coupling, Kf['coupling'], v, gc)
#    f_union = fittings(n_union, Kf['union'], v, gc)
#    f_gate_valve_full = fittings(n_gate_valve_full, Kf['gate_valve_full'], v, gc)
#    f_gate_valve_half = fittings(n_gate_valve_half, Kf['gate_valve_half'], v, gc)
#    
#    fittings_total = f_elbow45+f_elbow90+f_tee+f_return_bend+f_coupling+f_union+f_gate_valve_full+f_gate_valve_half
#    return fittings_total

def f_fittings (v, f_list, gc = 1): #set gc = 32.174 if english units
    Kf = {'elbow45': 0.45, 'elbow90': 0.75, 'tee': 1, 'return_bend': 1.5, \
         'coupling': 0.04, 'union': 0.04, 'gate_valve_full': 0.17, 'gate_valve_half': 4.5}
    #f_elbow90= n_elbow90* Kf['elbow90'] * v**2 * 0.5 / gc
    f_elbow45 = fittings(f_list[0], Kf['elbow45'], v, gc)
    f_elbow90 = fittings(f_list[1], Kf['elbow90'], v, gc)
    f_tee = fittings(f_list[2], Kf['tee'], v, gc)
    f_return_bend = fittings(f_list[3], Kf['return_bend'], v, gc)
    f_coupling = fittings(f_list[4], Kf['coupling'], v, gc)
    f_union = fittings(f_list[5], Kf['union'], v, gc)
    f_gate_valve_full = fittings(f_list[6], Kf['gate_valve_full'], v, gc)
    f_gate_valve_half = fittings(f_list[7], Kf['gate_valve_half'], v, gc)
    
    fittings_total = f_elbow45+f_elbow90+f_tee+f_return_bend+f_coupling+f_union+f_gate_valve_full+f_gate_valve_half
    return fittings_total

## Define head $ g dZ $

In [ ]:
def head (z_inlet, z_outlet, g=9.81, gc = 1):
    return g*(z_inlet - z_outlet)/gc

# Define $udu$

In [1]:
#def udu (v1, v2):
#    return (v1**2 - v2**2)/2

# Solve for Outlet Pressure

In [ ]:
def p_out (p1, f_total, gdz, udu, density, Q):
    return p1 + (udu + gdz - f_total + Q)*density*1e-6

In [ ]:
'''Solve for steam fraction at pipe outlet'''
def x_out(t_loss, p_in, p_out):
    steam = IAPWS97(P = p_in, x=1)
    Hg = steam.h
    steam = IAPWS97(P = p_out, x=1)
    hg = steam.h
    brine = IAPWS97(P = p_out, x=0)
    hf = brine.h
    #print(Hg, hg, hf, p_out, t_loss)
    return ((t_loss) + Hg - hf)/(hg-hf)

### References:

Geankoplis, C. J. (1993). Transport Process and Unit Operations, 3aEd.

Rajput, R.K. (1999). Heat and mass transfer: New Delhi, India, S. Chand and Company Ltd., 691 pp.

Verma, M. P. (2013). Steam transport simulation in a geothermal pipeline network constrained by internally consistent thermodynamic properties of water. Revista Mexicana de Ciencias Geológicas, 30(1), 210-221.